# Box Canyon Park 

https://www.google.com/maps/place/Box+Canyon+Park/@33.8857385,-117.7309815,910m/data=!3m2!1e3!4b1!4m6!3m5!1s0x80dcce2292288e59:0xc1684d99397ec327!8m2!3d33.8857385!4d-117.7284066!16s%2Fg%2F1tm8kwym?entry=ttu

## 1. Organize data

Create a folder under `semantic_SfM/data` and organize your data following the structures below. 

Using SfM products from Agisoft:
```
semantic_SfM/data
    ├── box_canyon_park
        ├── DJI_photos
        │       ├── DJI_0000.JPG
        │       ├── DJI_0001.JPG
        │       ├── ...
        │       └── DJI_0100.JPG
        ├── SfM_products
        │       ├── agisoft_cameras.xml
        │       ├── model.jpg
        │       ├── model.obj
        │       ├── model.mtl
        │       └── agisoft_model.las   
        ├── segmentations
        └── associations

```

In [1]:
import os

scene_dir = '../data/box_canyon_park'
pointcloud_path = os.path.join(scene_dir, 'SfM_products', 'model.las')
associations_folder_path = os.path.join(scene_dir, 'associations')
segmentations_folder_path = os.path.join(scene_dir, 'segmentations')
photos_folder_path = os.path.join(scene_dir, 'DJI_photos')
camera_path = os.path.join(scene_dir, 'SfM_products', 'agisoft_cameras.xml')
mesh_path = os.path.join(scene_dir, 'SfM_products', 'model.obj')

## 2. Create 2D Segmentation using SAM

In [2]:
from ssfm.image_segmentation import ImageSegmentation
import os

In [4]:
sam_params = {}
sam_params['model_name'] = 'sam2'
sam_params['model_path'] = '../semantic_SfM/sam2/sam2.1_hiera_large.pt'
sam_params['device'] = 'cuda:0'
sam_params['points_per_side'] = 32
sam_params['points_per_batch'] = 128
sam_params['pred_iou_thresh'] = 0.6
sam_params['stability_score_offset'] = 0.5
sam_params['box_nms_thresh'] = 0.6
sam_params['use_m2m'] = True


image_path_list = [os.path.join(photos_folder_path, image) for image in os.listdir(photos_folder_path)]

# sort images based on the values of keyimages in file names
image_path_list = sorted(image_path_list, key=lambda x: int(x.split('/')[-1].split('.')[0].split('_')[-1]))

image_list = [image for image in os.listdir(photos_folder_path)]

# sort images based on the values of keyimages in file names
image_list = sorted(image_list, key=lambda x: int(x.split('/')[-1].split('.')[0].split('_')[-1]))

In [ ]:
run_segmentation = False

if run_segmentation:
    image_segmentor = ImageSegmentation(sam_params)   
    image_segmentor.set_distortion_correction(camera_path)
    image_segmentor.batch_predict(image_path_list, segmentations_folder_path, save_overlap=True, skip_existing=False)

## 3. Create projection associations

In [5]:
from ssfm.probabilistic_projection import *
import time

In [6]:
pointcloud_projector = PointcloudProjection(depth_filtering_threshold=0.01)
#pointcloud_projector = PointcloudProjection()

In [7]:
pointcloud_projector.read_camera_parameters(camera_path)
pointcloud_projector.read_mesh(mesh_path)
pointcloud_projector.read_pointcloud(pointcloud_path)

In [8]:
pointcloud_projector.parallel_batch_project_joblib(image_list, associations_folder_path, num_workers=8)

Processing frames: 100%|██████████| 129/129 [04:46<00:00,  2.22s/it]


In [9]:
# build keyimage associations
from ssfm.keyimage_associations_builder import *

In [10]:
smc_solver = KeyimageAssociationsBuilder(image_list, associations_folder_path, segmentations_folder_path)
smc_solver.build_associations()
smc_solver.build_graph(10)

100%|██████████| 129/129 [00:50<00:00,  2.53it/s]


Building edges on GPU with 10 chunks took 5.454965114593506 seconds.


In [11]:
smc_solver.add_camera_to_graph([camera_path], camera_type="Agisoft")

../data/box_canyon_park/associations/graph_with_cameras.graphml


In [ ]:
smc_solver.find_min_cover()
#smc_solver.refine(0.5)

## 4. Estimate memory usage

In [12]:
from ssfm.memory_calculator import memory_calculator

In [29]:
# pointcloud file
las_file = pointcloud_path
# image file sample; this needs to be an original image even if patch images are used
image_file = os.path.join(photos_folder_path, image_list[0])
# number of images
num_images = len(image_list)
# number of segmentation ids for each point in the point cloud
num_segmentation_ids = 5

memory_calculator(las_file, image_file, num_images, num_segmentation_ids)

+----------------------------------------+----------------------+
|              Memory Type               | Memory Required (GB) |
+----------------------------------------+----------------------+
|      Segmentation for each image       | 0.037181854248046875 |
| Pixel2point association for each image | 0.07436370849609375  |
| Point2pixel association for each image | 0.17169087380170822  |
|                                        |                      |
|      Segmentation for all images       |  4.796459197998047   |
| Pixel2point association for all images |  9.592918395996094   |
| Point2pixel association for all images |  22.14812272042036   |
|          pc_segmentation_ids           |  0.8584543690085411  |
|         pc_segmentation_probs          |  0.8584543690085411  |
|          keyimage_association          |   5.53703068010509   |
|                 Total                  |  43.79143973253667   |
+----------------------------------------+----------------------+


## 5. Run object registration

In [13]:
from ssfm.object_registration import *
from ssfm.post_processing import *
import time

In [14]:
obr = ObjectRegistration(pointcloud_path, segmentations_folder_path, associations_folder_path, image_list=image_list, using_graph=True)

# Run object registration
obr.object_registration(iou_threshold=0.5, save_semantics=True)

Processing images: 100%|██████████| 129/129 [2:42:48<00:00, 75.72s/it]  


In [15]:
image_id = 128
semantics_folder_path = os.path.join(associations_folder_path, 'semantics', 'semantics_{}.npy'.format(image_id))
save_las_path = os.path.join(associations_folder_path, 'semantics', 'semantics_{}.las'.format(image_id))
add_semantics_to_pointcloud(pointcloud_path, semantics_folder_path, save_las_path, remove_small_N=500, nearest_interpolation=500)
#add_semantics_to_pointcloud(pointcloud_path, semantics_folder_path, save_las_path)

Before removing small semantics: 
maximum of semantics:  8922
number of unique semantics:  1693
After removing small semantics: 
number of unique semantics:  411


In [ ]:
print(save_las_path)

# read las 
pc = laspy.read(save_las_path)
# get the semantics from the intensity
semantics = pc.intensity
semantics_ids = np.unique(semantics)
print('number of unique semantics: ', len(semantics_ids))
# print the number of points for each semantics
for i in semantics_ids:
    n = np.sum(semantics == i)
    if n < 100:
        print('semantics id: ', i, ' number of points: ', n)

In [16]:
semantic_pc_file_path = save_las_path
post_processing = PostProcessing(semantic_pc_file_path)
post_processing.shuffle_semantic_ids(exclude_largest_semantic=False)
save_las_path = os.path.join(associations_folder_path, 'semantics', 'semantics_{}_shuffled.las'.format(image_id))
post_processing.save_semantic_pointcloud(save_las_path)

Number of unique semantics:  410
